# Create your own models
This Notebook illustrates how to create your own models using the framework. At the end of this guide, you'll be able to run simulations with your own models. For a guide on how to create new metrics, please see [advanced-metrics](advanced-metrics.ipynb). In what follows, we assume you are familiar with the main concepts of the framework shown in [complete-guide](complete-guide.ipynb).

## Dynamics
Recall that the dynamics of the framework are expressed by the following steps:
> 1. The **model** presents the **users** with some recommended **items**. In general, the items are chosen such that they maximize the probability of user engangement. This probability is based on the model's _prediction_ of user preferences.
> 2. The **users** view the items presented by the **model**, and interact with some **items** according to some _actual_ preferences.
> 3. The **model** updates its system state (such as the prediction of user preferences) based on the interactions of **users** with **items**, and it takes some **measurements**.

## Skeleton code
This code illustrates the skeleton to define a new model, NewModel:
```python
from trecs.models import BaseRecommender

class NewModel(BaseRecommender):
    def __init__(self, ...):
        # ...
        BaseRecommender.__init__(self, ...)
    
    def _update_internal_state(self, interactions):
        # ...

```

### `__init__`

1. The constructor must initialize a number of data structures to pass to the parent constructor. Importantly, it must initialize or pass:
    - An array of user preferences as predicted by the system (`users_hat`)
    - An array of item attributes in the system (`items_hat`)
    - A representation of "real" user profiles and item attributes (`users`, `items`) 
    - The number of users in the system (`num_users`)
    - The number of items in the system (`num_items`)
    - The number of items presented to each user at each time step (`num_items_per_iter`)
    - Any optional keyword arguments (such as metrics, system state components to be monitored, verbosity for logging, random seed, etc.)
    
   Any other class attribute (e.g., [infection state](https://elucherini.github.io/algo-segregation/reference/models.html#models.bass.InfectionState)) must be inizialized by `NewModel`.
       
2. The constructor must then call the `BaseRecommender` constructor respecting its signature.

#### Concrete example
Below we show an example of the `__init__` function, taken from the `PopularityRecommender` model. Note that we first conduct a series of input checks (which may be unnecessary if you are developing your own model purely for your own use). For the popularity recommender system, the recommender system's representation of item attributes is simply a vector of zeroes by default, implying that at the start of the simulation, all items have zero interactions and are thus equally popular. Of course, the user is free to pass in a different item representation; for example, passing in `item_representation=np.array([100,0,0])` would represent a scenario where one item was already much more popular than the others.

```python
# We define default values in the signature so we can call the constructor with no argument
def __init__(self, num_users=None, num_items=None, user_representation=None, item_representation=None, actual_user_representation=None, actual_item_representation=None, probabilistic_recommendations=False, seed=None, verbose=False, num_items_per_iter=10, **kwargs):
        num_users, num_items, num_attributes = validate_user_item_inputs(
            num_users,
            num_items,
            user_representation,
            item_representation,
            actual_user_representation,
            actual_item_representation,
            100,
            1250,
            num_attributes=1,
        )
        # num_attributes should always be 1
        if item_representation is None:
            item_representation = np.zeros((num_attributes, num_items), dtype=int)
        # if the actual item representation is not specified, we assume
        # that the recommender system's beliefs about the item attributes
        # are the same as the "true" item attributes
        if actual_item_representation is None:
            actual_item_representation = item_representation.copy()
        if user_representation is None:
            user_representation = np.ones((num_users, num_attributes), dtype=int)

        super().__init__(
            user_representation,
            item_representation,
            actual_user_representation,
            actual_item_representation,
            num_users,
            num_items,
            num_items_per_iter,
            probabilistic_recommendations=probabilistic_recommendations,
            verbose=verbose,
            seed=seed,
            **kwargs
        )
```

### `_update_internal_state`

This function is called at each timestep, right after the system has collected the interactions from users. In this step, we update the internal state of the system based on the user interactions. `interactions` is an array of size `num_users` in which element `u` is the index of the item that user `u` has interacted with.

So the necessary steps are:
1. The signature must be `_update_internal_state(self, interactions)`
2. It should not return anything; all necessary updates must be in the body of the function.


#### Concrete example
Still following [PopularityRecommender](https://elucherini.github.io/t-recs/reference/models.html#models.popularity.PopularityRecommender), this a possible implementation of `_update_internal_state`.

```python
# In the PopularityRecommender, we update item representations with the number of interactions they received in the last timestep
def _update_internal_state(self, interactions):
        histogram = np.zeros(self.num_items, dtype=int)
        np.add.at(histogram, interactions, 1)
        self.items_hat.value += histogram
```


### `score_fn`
This function takes a matrix of user profiles and item attributes and returns, for each user and item, the predicted "score," where a score roughly represents the system's prediction of the given user's propensity to interact with the given item. By default, the `score_fn` is simply the inner product. Depending on how the user and item representations are constructed, the inner product is able to recover a large number of possible recommender system algorithms. However, you are free to overwrite the `score_fn` attribute by passing in a different function to the `BaseRecommender` initialization function.

Here is an example:

```python
from trecs.models import BaseRecommender
import numpy as np
import trecs.matrix_ops as mo

class NewModel(BaseRecommender):
    def __init__(self, ...):
        # ...
        BaseRecommender.__init__(self, score_fn=self.cosine_similarity, ...)
    
    def _update_internal_state(self, interactions):
        # ...
    
    def cosine_similarity(self, user_profiles, item_attributes):
        """
        Calculate cosine similarity for each user, item pair.
        """
        denominator = np.outer(np.linalg.norm(user_profiles, axis=1), np.linalg.norm(item_attributes, axis=0))
        # cosine similarity is equal to inner product, divided by the norm of the user & item vector
        return mo.inner_product(user_profiles, item_attributes) / denominator
```


In [8]:
from trecs.validate import validate_user_item_inputs
import numpy as np
from trecs.models import BaseRecommender
from trecs.random import Generator


class NewModel(BaseRecommender):
    def __init__(self, num_users=None, num_items=None, user_representation=None, item_representation=None, actual_user_representation=None, actual_item_representation=None, probabilistic_recommendations=False, seed=None, verbose=False, num_items_per_iter=10, **kwargs):
        num_users, num_items, num_attributes = validate_user_item_inputs(
            num_users,
            num_items,
            user_representation,
            item_representation,
            actual_user_representation,
            actual_item_representation,
            100,
            1250,
            num_attributes=1,
        )
        # num_attributes should always be 1
        if item_representation is None:
            item_representation = np.zeros((num_attributes, num_items), dtype=int)
        # if the actual item representation is not specified, we assume
        # that the recommender system's beliefs about the item attributes
        # are the same as the "true" item attributes
        if actual_item_representation is None:
            actual_item_representation = item_representation.copy()
        if user_representation is None:
            user_representation = np.ones((num_users, num_attributes), dtype=int)

        super().__init__(
            user_representation,
            item_representation,
            actual_user_representation,
            actual_item_representation,
            num_users,
            num_items,
            num_items_per_iter,
            probabilistic_recommendations=probabilistic_recommendations,
            verbose=verbose,
            seed=seed,
            **kwargs
        )
    
    def _update_internal_state(self, interactions):
        histogram = np.zeros(self.num_items, dtype=int)
        np.add.at(histogram, interactions, 1)
        self.items_hat.value += histogram

## And now let's use it to run a simulation:

In [9]:
model = NewModel(num_users=1500)
model.run(timesteps=10)

/mnt/c/Users/matth/pton-research/recsys/t-recs/trecs/models/recommender.py:662: UserWarning: No measurements are currently defined for the simulation. Please add measurements if desired.
  warnings.warn(error_msg)
100%|██████████| 10/10 [00:09<00:00,  1.06it/s]
